<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/scenario_oncology_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create fictive clinical notes from Code set (DRG + ICD)

Code set are the raw classification data, we can extract from National database (Base nationale PMSI en France). They are made of 
* classification profile made of grouping variables from DRG records which are prepared with their frequency in the national database
    - age (class)
    - sexe
    - DRG (racine GHM)
    - Main diagnosis (ICD10) : cf
    - Hospitalization management type : cf
* diagnosis associated to each classification profile, extracted with their frequencies
* procedures associated to each classification profile, specialy for surgery and technical gestures, extracted with their frequencies

From thoses raw information we produce a coded clinical scenario which will be uses a seed.

This scenario is transformed into a detail prompt that will be given to a LLM for generation.
From the combinaision of primary and related diagnosis in French discharge abstract, we derived two notions :
* Primary diagnosis : host the notion of principal pathology, it is rather the primary diagnosis of the discharge abstract or the related diagnosis when it exists and that the primary diagnosis of the discharge abstract is from the chapter "Facteurs influant sur l’état de santé" of ICD10
* The Hospitalization management type is rather the term "Primary diagnosis" or the ICD-10 code of the related diagnosis when it exists


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
from utils import *

In [147]:
gs = generate_scenario()
# Load official dictionaries
# col_names option allow you to algin your column names the project dictionary.
gs.load_offical_icd("cim_2024.xlsx",col_names={"code" : "icd_code","libelle":"icd_code_description"} )
gs.load_offical_procedures("ccam_actes_2024.xlsx",col_names={"code":"procedure","libelle_long":"procedure_description"} )
col_names={"Code CIM":"icd_parent_code","Localisation":"primary_site","Type Histologique":"histological_type",
	"Stade":"stage","Marqueurs Tumoraux":"biomarkers","Traitement":"treatment_recommandation","Protocole de Chimiothérapie":"chemotherapy_regimen"}
gs.load_cancer_treatement_recommandations("Tableau récapitulatif traitement cancer.xlsx",col_names ) 

In [148]:
# Load data from BN  PMSI
col_names={"racine":"drg_parent_code","das": "icd_secondary_code","diag":"icd_primary_code","categ_cim":"icd_primary_parent_code",
            "mdp":"case_management_type","nb_situations":"nb","acte":"procedure",
            "mode_entree":"admission_mode",
            "mode_sortie":"discharge_disposition",
            "mode_hospit":"admission_type"}
gs.load_classification_profile("bn_pmsi_cases_20250819.csv", col_names)
gs.load_secondary_icd("bn_pmsi_related_diag_20250818.csv",col_names)
gs.load_procedures("bn_pmsi_procedures_20250818.csv",col_names)

In [235]:
query = "icd_primary_code=='C50' and case_management_type=='Z511'"
current_profile = gs.df_classification_profile.query(query).drop(columns="nb").iloc[1]
current_profile

icd_primary_code                                             C50
case_management_type                                        Z511
drg_parent_code                                            28Z07
age                                                        ge_18
cage                                                     [30-40[
cage2                                                    [18-50[
sexe                                                           2
admission_type                                         Inpatient
admission_mode                                          DOMICILE
discharge_disposition                                   DOMICILE
dms                                         1,32352941176471e+00
los_mean                                                     2.0
los_sd                                                       0.0
drg_parent_description    Chimiothérapie pour tumeur, en séances
da                                                           D27
libelle_da               

In [273]:
current_profile["icd_parent_code"] = current_profile["icd_primary_code"][0:3]

scenario = gs.get_clinical_scenario_template()

is_cancer = 0
for k,v in current_profile.items():
    scenario[k]=v 

### Principals diagnosis
scenario["icd_primary_description"] = gs.get_icd_description(current_profile.icd_primary_code)

scenario["icd_primary_description_alternative"] = gs.get_icd_alternative_descriptions(current_profile.icd_primary_code)
scenario["case_management_type_description"] = gs.get_icd_description(current_profile.case_management_type)

### Administratives elements
scenario["age"] = get_age(current_profile.cage)
scenario["date_entry"],scenario["date_discharge"] = get_dates_of_stay(current_profile.admission_type,current_profile.admission_mode,current_profile.los_mean,current_profile.los_sd)
scenario["date_of_birth"] = random_date_between( scenario["date_entry"] - datetime.timedelta(days = 365*(scenario["age"]+1)) , scenario["date_entry"] - datetime.timedelta(days = 365*(scenario["age"])))
scenario["first_name"] , scenario["laste_name"] = gs.get_names(current_profile.sexe)
scenario["first_name_med"] , scenario["laste_name_med"] = gs.get_names(random.randint(1, 2))


### Secondary diagnosis :
### We sample secondary diagnosis by steps : metastases, metastases ln, chronic,complications
### Each time build :
### - official descriptions
### - official : alternatives descripttion
scenario["text_secondary_icd_official"]=""
scenario["text_secondary_icd_alternative"]=""

grouping_secondary =["icd_primary_code","icd_secondary_code","cage2","sexe","nb"] 

### Scenarios will be much more different when clinical case is about cancer
if scenario["icd_primary_code"] in gs.icd_codes_cancer:
  is_cancer =1
  ## When care pathway is related to the treatment of cancer and when recommandations are availabe, the clinical case will be create from those recommandation
  if scenario["case_management_type"] in ['DP','Z511'] : 
    #treatment_recommendations = gs.sample_from_df(profile =pd.Series({"icd_parent_code":"C50"}),df_values= gs.df_cancer_treatment_recommandation,nb=1) 
    
    treatment_recommendations = gs.sample_from_df(profile =current_profile,df_values= gs.df_cancer_treatment_recommandation,nb=1) 
    if treatment_recommendations.shape[0]>0 :
      scenario["histological_type"] = treatment_recommendations["histological_type"].iloc[0] 
      score_TNM = treatment_recommendations["TNM"].iloc[0]
      if score_TNM not in ['Variable','Non pertinent'] :
        scenario["score_TNM"] = score_TNM
      stage = treatment_recommendations["stage"].iloc[0]
      if stage not in ['Variable','Non pertinent'] :
        scenario["cancer_stage"] = stage
      
      scenario["treatment_recommandation"] = treatment_recommendations["treatment_recommandation"].iloc[0] 
      if treatment_recommendations.chemotherapy_regimen.notna().bool:
        scenario["chemotherapy_regimen"] = treatment_recommendations["chemotherapy_regimen"].iloc[0]  

      scenario["biomarkers"] = treatment_recommendations["biomarkers"].iloc[0]
    
    ### Categories of chronic desease : when cancer you don't sample chronic diseases over cancer icd codes


### Treatment recommandations for cancer :
if( is_cancer ==1 ) :
     chronic_diseases = gs.sample_from_df(profile =current_profile,df_values= gs.df_secondary_icd.query("type.isin(['Chronic'])")[grouping_secondary])

else :     
  chronic_diseases =  gs.sample_from_df(profile =current_profile,df_values= gs.df_secondary_icd.query("type.isin(['Chronic'])")[grouping_secondary])  

if chronic_diseases.shape[0] > 0 :
    for index, row in chronic_diseases.iterrows():
        scenario["text_secondary_icd_official"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+")\n"
        scenario["text_secondary_icd_alternative"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+") : " + row.icd_code_description_alternative + "\n"
    
    scenario["chronic"] = chronic_diseases.icd_secondary_code.to_list()

### Is we sampled cancer codes in the chronic disease we will also sample metastasis
if scenario["chronic"] is not None:
  if bool(set(scenario["chronic"]) & set(gs.icd_codes_cancer)) :
    is_cancer = 1
    

### LN metastasis
if is_cancer ==1 :

  ##If TNM is know sample metastasis regarding this status
  if scenario["TNM_score"] is not None :
    if bool(re.search("N[123x]",scenario["TNM_score"])) :
      metastases_ln = gs.sample_from_df(profile =current_profile,df_values= gs.df_secondary_icd.query("type=='Metastasis LN'")[grouping_secondary],nb = 1)  
      if metastases_ln.shape[0] > 0 :
        for index, row in metastases_ln.iterrows():
          scenario["text_secondary_icd_official"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+")\n"
          scenario["text_secondary_icd_alternative"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+") : " + row.icd_code_description_alternative + "\n"
    
        scenario["chronic"] = scenario["chronic"] + metastases_ln.icd_secondary_code.to_list()

    if bool(re.search("M[123x]",scenario["TNM_score"])) :
      metastases = gs.sample_from_df(profile =current_profile,df_values= gs.df_secondary_icd.query("type=='Metastasis'")[grouping_secondary])  
      if metastases.shape[0] > 0 :
        for index, row in metastases.iterrows():
          scenario["text_secondary_icd_official"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+")\n"
          scenario["text_secondary_icd_alternative"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+") : " + row.icd_code_description_alternative + "\n"
        
        scenario["chronic"] = scenario["chronic"] + metastases.icd_secondary_code.to_list()
    
    
    #When TNM is not known, sample metastasis among all possible situations
    else:
      metastases = gs.sample_from_df(profile =current_profile,df_values= gs.df_secondary_icd.query("type.isin['Metastasis','Metastasis LN']")[grouping_secondary])  
      if metastases.shape[0] > 0 :
        for index, row in metastases.iterrows():
          scenario["text_secondary_icd_official"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+")\n"
          scenario["text_secondary_icd_alternative"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+") : " + row.icd_code_description_alternative + "\n"
        
        scenario["chronic"] = scenario["chronic"] + metastases.icd_secondary_code.to_list()

#For complication drg_parent_code we choose grouping profile only on ICD
grouping_secondary =["drg_parent_code","icd_secondary_code","cage2","sexe","nb"]

complications = gs.sample_from_df(profile =current_profile,df_values= gs.df_secondary_icd.query("type.isin(['Acute'])")[grouping_secondary]) 

if complications.shape[0] > 0 :
  for index, row in complications.iterrows():
    scenario["text_secondary_icd_official"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+")\n"
    scenario["text_secondary_icd_alternative"] += "- " + row.icd_code_description_official + "("+ row.icd_secondary_code+") : " + row.icd_code_description_alternative + "\n"
  
  scenario["chronic"] = scenario["chronic"] + complications.icd_secondary_code.to_list()

## Actes

grouping_secondary =["procedure","drg_parent_code","icd_primary_code","cage2","sexe"]
procedures = gs.sample_from_df(profile =current_profile,df_values= gs.df_procedures[grouping_secondary],nb=1) 

scenario["procedure"] = procedures.procedure.to_list()

scenario["text_procedure"] = ""

for index, row in procedures.iterrows():
          scenario["text_procedure"] += "- " + row.procedure_description_official + "("+ row.procedure+")\n"


scenario["case_management_type_text"] , scenario["cd_md_pec"] = gs.define_cancer_md_pec(scenario)




'T2N1M0'

In [275]:
SCENARIO = "**SCÉNARIO DE DÉPART :**\n"

for k,v in scenario.items():
  if k == "age" and v is not None:
    SCENARIO +="- Âge du patient : " + str(v) + "ans\n"
  if k == "sexe" and v is not None:  
    SCENARIO +="- Sexe du patient : " + str(v) + "\n"
  if k == "date_entry" and v is not None:  
    SCENARIO +="- Date d'entrée : "+ v.strftime("%d/%m/%Y") + "\n"
  if k == "date_discharge" and v is not None:  
    SCENARIO +="- Date de sortie : "+ v.strftime("%d/%m/%Y") + "\n" 
  if k == "date_of_birth" and v is not None:  
    SCENARIO +="- Date de naissance : "+ v.strftime("%d/%m/%Y") + "\n"  
  if k == "last_name" and v is not None:  
    SCENARIO +="- Nom du patient : "+ v + "\n" 
  if k == "first_name" and v is not None:  
    SCENARIO +="- Prénom du patient : "+ v + "\n" 
  if(scenario["icd_primary_code"] in gs.icd_codes_cancer) : 
    if k == "icd_primary_code_definition" and v is not None:  
      SCENARIO +="- Localisation anatomique de la tumeur primaire : ("+ v + scenario["icd_primary_code"] + ")\n" 
    if k == "histological_type":
      if v is not None:  
          SCENARIO +="- Type anatomopathologique de la tumeur primaire : ("+ v + ")\n" 
      else :
          SCENARIO +="- Type anatomopathologique de la tumeur primaire : Vous choisirez vous même un type histologique cohérent avec la localisation anatomique\n" 
    if k == "score_TNM":
      if v is not None:  
          SCENARIO +="- Score TNM :"+ v + "\n" 
      else :
          SCENARIO +="- Score TNM : Si la notion de score de TNM est pertinente avec le type histologique et la localisation anatomique, vous choisirez un score TNM\n" 
    if k == "cancer_stage":
       if v is not None:  
          SCENARIO +="- Stade tumoral : " + v + "\n" 
    if k == "biomarkers":
       if v is not None:  
          SCENARIO +="- Biomarqueurs tumoraux : "+ v + "\n" 
       else :
          SCENARIO +="- Biomarqueurs tumoraux : Vous choisirez des biomarqueur tumoraux cohérent avec la localisation et l'histologie de la tumeur\n" 
  
  if k == "mode_entree" and v is not None:  
    SCENARIO +="- Mode d'entrée' : "+ v + "\n" 
  if k == "mode_sortie" and v is not None:  
    SCENARIO +="- Mode de sortie' : "+ v + "\n" 

  if k == "case_management_type" and v is not None:  
    SCENARIO +="- Mode de prise en charge : "+ scenario["case_management_type_text"] + "\n" 
    SCENARIO +="- codage CIM10 :\n" 
    SCENARIO +="   * Diagnostic principal : "+  scenario["icd_primary_description"] + "("+ scenario["icd_primary_code"] + ")\n"  
    if scenario["case_management_type"]!="DP":
      SCENARIO +="   * Diagnostic relié : "+  scenario["case_management_type_description"] + "("+ scenario["case_management_type"] + ")\n"  
    else:
      SCENARIO +="   * Diagnostic relié :  Aucun\n"  
    
    SCENARIO +="   * Diagnostic associés : \n"
    SCENARIO +=  scenario["text_secondary_icd_official"]  + "\n"  

print(SCENARIO)


**SCÉNARIO DE DÉPART :**
- Âge du patient : 37ans
- Sexe du patient : 2
- Date d'entrée : 09/07/2024
- Date de sortie : 11/07/2024
- Date de naissance : 09/10/1986
- Prénom du patient : Louisette
- Mode de prise en charge : Hospitalisation pour prise en charge du cancer : vous choisirez un épisode de traitement sachant que les recommandations pour ce stade du cancer sont les suivantes :
 - Schéma thérapeutique : Chirurgie (mastectomie ou BCS) + radiothérapie, thérapie systémique adjuvante incluant chimiothérapie, thérapie ciblée HER2 si HER2+
 - Protocole de chimiothérapie : FEC (5-Fluorouracile, Épirubicine, Cyclophosphamide) suivi de Taxane

- codage CIM10 :
   * Diagnostic principal : Tumeur maligne du sein(C50)
   * Diagnostic relié : Séance de chimiothérapie pour tumeur(Z511)
   * Diagnostic associés : 
- Sclérose en plaques(G35)
- Surveillance et conseils diététiques(Z713)
- Isolement (prophylactique)(Z290)
- Personne consultant pour le compte d'un tiers(Z710)
- Antécédents perso

In [276]:

ICD_ALTERNATIVES =""

if scenario["icd_primary_code"] not in gs.icd_codes_cancer :
      ICD_ALTERNATIVES +=" - " + scenario["icd_primary_description"] + "("+ scenario["icd_primary_code"] + ") : " 
      ICD_ALTERNATIVES +=": "+ scenario["icd_primary_description_alternative"] + "\n"
ICD_ALTERNATIVES +=  scenario["text_secondary_icd_alternative"]  + "\n" 

print(ICD_ALTERNATIVES)


- Sclérose en plaques(G35) : sep disseminee, sclerose insulaire, sep generalisee, sep forme progressive, sclerose multiloculaire multiple
- Surveillance et conseils diététiques(Z713) : education dietetique pour obesite, surveillance et conseils dietetiques, surveillance dietetique pour diabete sucre, patient venant pour conseil dietetique, education dietetique pour diabete sucre
- Isolement (prophylactique)(Z290) : isolement cutane, isolement prophylactique, malade isole en chambre sterile, isolement, mesure prophylactique isolement
- Personne consultant pour le compte d'un tiers(Z710) : personne en bonne sante consultant pour une tierce personne, personne consultant pour le compte d un tiers, personne en consultation pour une tierce personne, conseil tierce personne
- Antécédents personnels de COVID-19, sans précision(U089) : Antécédents personnels de COVID-19, sans précision




In [284]:
INSTRUCTIONS_CANCER = ""
if scenario["histological_type"] is not None:
    INSTRUCTIONS_CANCER ="vous choisirez un épisode de traitement sachant que les recommandations pour ce stade du cancer sont les suivantes :\n"
    INSTRUCTIONS_CANCER +=" - Schéma thérapeutique : " + scenario["treatment_recommandation"] + "\n"
    if scenario["chemotherapy_regimen"] is not None :
        INSTRUCTIONS_CANCER += " - Protocole de chimiothérapie : " + scenario["chemotherapy_regimen"] 

    INSTRUCTIONS_CANCER += "Veillez à bien préciser le type histologique et la valeur des biomarqueurs si recherchés\n"

In [288]:
def prepare_prompt(prompt_path, case):
  with open(prompt_path, "r", encoding="utf-8") as f:
      content = f.read()
  return (content
          .replace("[SCENARIO here]", case["SCENARIO"])
          .replace("[INSTRUCTIONS_CANCER here]", case["INSTRUCTIONS_CANCER"])
          .replace("[ICD_ALTERNATIVES here]", case["ICD_ALTERNATIVES"])

          )

print(prepare_prompt("templates/scenario_onco_v1.txt",case = {"SCENARIO":SCENARIO,"INSTRUCTIONS_CANCER":INSTRUCTIONS_CANCER,"ICD_ALTERNATIVES":ICD_ALTERNATIVES}))

Vous êtes un oncologue clinicien expert. Votre tâche est de générer un compte rendu d'hospitalisation en style clinique synthétique à partir d'un scénario comprenant le résumé PMSI (codes de la classification internationale des maladies), ainsi que d'autres informations décrivant l'hospitalisation.


**SCÉNARIO DE DÉPART :**
- Âge du patient : 37ans
- Sexe du patient : 2
- Date d'entrée : 09/07/2024
- Date de sortie : 11/07/2024
- Date de naissance : 09/10/1986
- Prénom du patient : Louisette
- Mode de prise en charge : Hospitalisation pour prise en charge du cancer : vous choisirez un épisode de traitement sachant que les recommandations pour ce stade du cancer sont les suivantes :
 - Schéma thérapeutique : Chirurgie (mastectomie ou BCS) + radiothérapie, thérapie systémique adjuvante incluant chimiothérapie, thérapie ciblée HER2 si HER2+
 - Protocole de chimiothérapie : FEC (5-Fluorouracile, Épirubicine, Cyclophosphamide) suivi de Taxane

- codage CIM10 :
   * Diagnostic principal : T

In [270]:
print('Vous êtes un oncologue clinicien expert. Votre tâche est de générer un compte rendu d\'hospitalisation en style clinique synthétique à partir d\'un scénario comprenant le résumé PMSI (codes de la classification internationale des maladies), ainsi que d\'autres informations décrivant l\'hospitalisation.\n\n\n**SCÉNARIO DE DÉPART :**\n- Âge du patient : 37ans\n- Sexe du patient : 2\n- Date d\'entrée : 12/08/2024\n- Date de sortie : 14/08/2024\n- Date de naissance : 20/03/1987\n- Prénom du patient : Lisandre\n- Mode de prise en charge : Prise en charge en hospitalisation complète pour cure de chimiothérapie.Le protocole actuellement suivi est : AC (Doxorubicine/Cyclophosphamide) suivi de Taxane\n- codage CIM10 :\n   * Diagnostic principal : Tumeur maligne du sein(C50)\n   * Diagnostic relié : Séance de chimiothérapie pour tumeur(Z511)\n   * Diagnostic associés : \n- Hypertension essentielle (primitive)(I10)\n- Sclérose en plaques(G35)\n- Schizophrénie, sans précision(F209)\n- Anorexie mentale(F500)\n- Maladie de Crohn du gros intestin(K501)\n- Surveillance et conseils diététiques(Z713)\n- Séance de transfusion de produit sanguin labile(Z5130)\n- Pansinusite (chronique)(J324)\n\n- Stade tumoral : Stade I\n- Type anatomopathologique de la tumeur primaire : (Carcinome invasif (Ductal/NST, Lobular, Mixed, Micropapillary))\n- Score TNM :T1N0M0\n- Biomarqueurs tumoraux : ER/PR, HER2\n\n\n\n\n\n**INSTRUCTIONS :**\n\n- Vous devez utiliser exclusivement les diagnostics fournis. Aucun autre diagnostic ne doit être ajouté ou inventé.\t\t\n\n- Lors vous mentionnerez dans le texte les diagnostics, vous utiliserez autant que faire ce peut une formulation moins formelle que la définition du code. Pour vous aider nous vous proposons ci-dessous quelques exemples de formulation alternative qui peut être retrouvée dans les comptes rendus d\'hospitalisation.\n- Hypertension essentielle (primitive)(I10) : hypertension, hypertension arterielle severe, hypertensif, hypertension arterielle maligne, hta instable\n- Sclérose en plaques(G35) : sclerose multiloculaire, sclerose en plaques, demence sclerose en plaques, sep forme progressive, nevrite retrobulbaire sclerose en plaques\n- Schizophrénie, sans précision(F209) : psychose schizophrenique, etat schizophrenique chronique, schizophrenie infantile, schizophrenie familiale, schizophrenie vieillie\n- Anorexie mentale(F500) : anorexie psychologique, anorexie psychogenique, anorexie mentale masculine, anorexie reactionnelle, anorexie psychiatrique\n- Maladie de Crohn du gros intestin(K501) : maladie de crohn du gros intestin, maladie de crohn colique, crohn colique, crohn colo anal, maladie de crohn sigmoide\n- Surveillance et conseils diététiques(Z713) : education diabetique, education dietetique pour diabete sucre, personne en quete de conseils dietetiques, surveillance de l allaitement artificiel en dietetique infantile, patient venant pour conseil dietetique\n- Séance de transfusion de produit sanguin labile(Z5130) : seance de transfusion de produit sanguin labile\n- Pansinusite (chronique)(J324) : polysinusite infectieuse, pansinusite, pansinusite infectieuse, pansinusite chronique\n\n\n\n- Vous pourrez reformulez librement ces propositions de manière fluide et naturelle dans le texte. Néanmoins, vous devrez donner à la fin sous forme structuré, pour chaque diagnostic les formulations que vous aurez choisi sous la forme :\nCode CIM - Définition du code : formulations utilisées dans le texte.\n\n- Aucune partie du texte ne doit être mise en gras.\n\n- Langue : Rédigez le compte rendu en français.\n\n- Style : Adoptez un style clinique informel, tel qu\'utilisé couramment dans les notes cliniques quotidiennes.\n+ Ton télégraphique.\t\n+ Phrases éventuellement incomplètes (absence de sujet ou verbe) si cela reste naturel.\n+ Informations enchaînées par ponctuation simple : virgules, tirets, points-virgules, etc.\n\n- Formulation : \t\n+ Variez la structure des phrases. \t\n+ Variez les termes médicaux (synonymes, reformulations naturelles) tout en conservant leur précision clinique. Évitez les répétitions mot à mot.\t\n+ Utilisez, de manière occasionnelle et pertinente, des acronymes et abréviations médicales françaises courantes, sans en abuser.\n\n- Contenu à produire : Le compte rendu doit comporter les sections suivantes (max. 200 mots/section) :\t\n+ Motif d’hospitalisation\n+ Antécédents (médicaux, chirurgicaux, familiaux, allergies)\n+ Mode de vie (toxiques, profession, autres) par exemple activité physique, nombre de gestités et parités chez les femmes, situation sociale...)\n+ Histoire de la maladie\n+ Examen clinique (constantes vitales, examen physique, poids en kgs en particulier si une dénutrition est évoquée, etc.) \n+ Examens complémentaires avec résultats (uniquement ceux réalisés durant l’hospitalisation)\n+ Évolution pendant l\'hospitalisation: Veillez à être le plus exhaustif possible, en particulier indiquez : Conclusions des principaux examens complémentaires réalisés dans la section « Examens complémentaires »; Prise en charge thérapeutique; Suivi mis en place; Critères de sortie\n+ Conclusion\n\n- Chaque section doit commencer par un titre au format suivant :\n### Nom de la section (ex. : ### Motif d\'hospitalisation)\n- Attribuez des dates plausibles et cohérentes aux événements majeurs (hospitalisations, diagnostics, examens, traitements...).\n- Assurez la cohérence temporelle et médicale entre les différentes sections.\n- Ne pas inclure : Clauses de non-responsabilité.\n- Veillez à être le plus détaillé possible concernant le cancer. Incluez, si pertinent, les informations suivantes : \n+ État général : score de performance explicite selon l’échelle ECOG/OMS\n+ Statut gestationnel (contraception, grossesse ou allaitement) : uniquement chez les femmes\n+ Évolutivité tumorale : si elle est mentionée, vous devez obligatoirement :\n* indiquer les examens ayant permis d’évaluer cette évolution, avec leurs résultats\n* faire référence explicite aux examens antérieurs utilisés comme points de comparaison, afin de démontrer une progression ou une stabilité de la tumeur dans le temps\n+ Type de traitements antitumoraux reçus\n+ Lignes de traitement antitumoral reçues: indiquez explicitement à quelle ligne de traitement le patient se trouve actuellement.\n+ La tolérance aiguë à la chimiothérapie\n\nVeillez à intégrer ces informations de façon naturelle dans le compte rendu, sans recourir à des formats semi-structurés, par exemple préférez une rédaction naturelle plutôt que « État général : ECOG 1 »).\n\n- À la fin du texte généré, fournissez :\n+ un dictionnaire listant les formulations utilisées pour chaque diagnostic fourni (y compris le diagnostic principal et les diagnostics associés) : {"Code CIM - Définition du code": ["formulation utilisée 1", "formulation utilisée 2", ...],...}\n* Par exemple : {"poussee hypertension arterielle": ["poussée hypertension arterielle", "poussée HTA"],...} \n+ un dictionnaire listant les segments textuels utilisés pour décrire les informations suivantes, reproduits strictement à l’identique (copiés-collés depuis le texte généré, sans aucune modification ni interprétation, y compris ponctuation et orthographe) :\n* Âge\n* Sexe\n* État général\n* Statut gestationnel (s\'il s\'agit d\'une femme)\n* Gestité\n* Comorbidités\n* Topographie de la tumeur primaire: Localisation anatomique de la tumeur primaire- Histologie tumorale : Type cellulaire explicite de la tumeur primaire. Ne pas inclure de localisations anatomiques générales (ex. : cancer du sein, tumeur pulmonaire...)\n* Stade tumoral\n* Biomarqueurs tumoraux\n* Sites métastatiques : Localisations anatomiques des métastases\n* Évolutivité tumorale\n* Examens pour diagnostic initial: Noms des examens qui ont permis d\'établir le diagnostic initial de la tumeur primaire\n* Examens d\'évaluation du stade tumoral : Noms des examens ayant servi à déterminer le stade tumoral initial- Examens d’évaluation de l’évolutivité tumorale : Noms des examens ayant permis de juger de la progression ou de la stabilité de la maladie dans le temps - Traitements antitumoraux : Type de traitements antitumoraux reçus, y compris les types de chirurgie pratiqués- Lignes de traitement antitumoral\n* Médicaments antitumoraux : Noms des médicaments antitumoraux administrés\n* Cycles de chimiothérapie\n* NFS: résultats explicites (valeurs ou qualificatifs comme "normal", "élevé",...)\n* Créatinine : résultats explicites (valeurs ou qualificatifs comme "normal", "élevé",...)\n* Bilan hepatique: résultats explicites (valeurs ou qualificatifs comme "normal", "élevé",...)\n\nPar exemple : {"Âge": ["55 ans"],"Stade tumoral" : ["cT4N1M2"],"Examens pour diagnostic initial": ["TDM thoracique", "IRM cérébrale"],...}- La sortie doit être un objet JSON au format suivant :{"CR": "le text généré","formulations": {  "diagnostics": {...},  "informations": {...}  }}Par exemple : {"CR": "Texte du compte rendu ici...","formulations": {  "diagnostics": {"poussee hypertension arterielle": ["poussée hypertension arterielle", "poussée HTA"]},  "informations": {"Âge": ["55 ans"], "Stade tumoral" : ["cT4N1M2"], "Examens pour diagnostic initial": ["TDM thoracique", "IRM cérébrale"]}  }}'
)

Vous êtes un oncologue clinicien expert. Votre tâche est de générer un compte rendu d'hospitalisation en style clinique synthétique à partir d'un scénario comprenant le résumé PMSI (codes de la classification internationale des maladies), ainsi que d'autres informations décrivant l'hospitalisation.


**SCÉNARIO DE DÉPART :**
- Âge du patient : 37ans
- Sexe du patient : 2
- Date d'entrée : 12/08/2024
- Date de sortie : 14/08/2024
- Date de naissance : 20/03/1987
- Prénom du patient : Lisandre
- Mode de prise en charge : Prise en charge en hospitalisation complète pour cure de chimiothérapie.Le protocole actuellement suivi est : AC (Doxorubicine/Cyclophosphamide) suivi de Taxane
- codage CIM10 :
   * Diagnostic principal : Tumeur maligne du sein(C50)
   * Diagnostic relié : Séance de chimiothérapie pour tumeur(Z511)
   * Diagnostic associés : 
- Hypertension essentielle (primitive)(I10)
- Sclérose en plaques(G35)
- Schizophrénie, sans précision(F209)
- Anorexie mentale(F500)
- Maladie de